# پیش پردازش
ابتدا کتابخانه های مورد نیاز رو وارد میکنیم

In [1]:
import pandas as pd
import numpy as np

توی دیتا فریم های خودشونه csv حالا نوبت خوندن فایل های :

In [2]:
movies_df = pd.read_csv('./datasets/movies.csv')
ratings_df = pd.read_csv('./datasets/ratings.csv')
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


حذف میکنیم و در ستون سال جدید ذخیره میکنیم title با استفاده از تابع جایگزینی پانداس سال رو از ستون

In [3]:
# ساختن ستون سال
movies_df['year'] = movies_df['title'].str.extract('(\d\d\d\d)', expand=True)
# title حذف کردن سال از ستون
movies_df['title'] = movies_df.title.replace(to_replace='(\(\d\d\d\d\))', value='', regex=True)
# خلاص شدن از شر هر کاراکتر فضای خالی انتهایی که ممکن است ظاهر شده باشد 
movies_df['title'] = movies_df['title'].str.strip()

movies_df.head()

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\('
<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\('
C:\Users\Pars\AppData\Local\Temp\ipykernel_3908\3128716998.py:2: SyntaxWarning: invalid escape sequence '\d'
  movies_df['year'] = movies_df['title'].str.extract('(\d\d\d\d)', expand=True)
C:\Users\Pars\AppData\Local\Temp\ipykernel_3908\3128716998.py:4: SyntaxWarning: invalid escape sequence '\('
  movies_df['title'] = movies_df.title.replace(to_replace='(\(\d\d\d\d\))', value='', regex=True)


,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


مقادیر موجود در ستون ژانرها رو به لیستی از ژانرها تبدیل میکنیم تا برای استفاده در آینده ساده تر بشه

In [4]:
# ژانر ها رو از هم جدا میکنیم split با استفاده از متد 
movies_df['genres'] = movies_df['genres'].str.split('|')
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


استفاده میکنیم One Hot Encoding از اونجایی که حفظ ژانرها در قالب فهرست برای تکنیک سیستم توصیه مبتنی بر محتوا بهینه نیست از تکنیک

در این مورد ما هر ژانر مختلف را در ستون‌هایی ذخیره می‌کنیم که شامل 1 یا 0 است. 1 نشان می‌دهد که یک فیلم آن ژانر را دارد و 0 نشان می‌دهد که ندارد

In [5]:
# با توجه به عدم نیاز استفاده از اطلاعات ژانر در مورد اول ما جدول داده فیلم را در یک جدول جدید کپی می‌کنیم
moviesWithGenres_df = movies_df.copy()

# برای هر ردیف در دیتا فریم در لیست ژانرها حرکت میکنیم و هر ژانر را در ستون مجزا قرار میدهیم و عدد 1 را در ستون مربوطه قرار می دهیم
for index, row in movies_df.iterrows():
    for genre in row['genres']:
        moviesWithGenres_df.at[index, genre] = 1
# (است NaN به اصطلاح مقدار آنها)پر کردن مقادیر با 0 برای نشان دادن اینکه یک فیلم آن ژانر ستون را ندارد
moviesWithGenres_df = moviesWithGenres_df.fillna(0)
moviesWithGenres_df.head()

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


خب حالا بیاید به دیتا فریم امتیازات یه نگاهی بندازیم

In [6]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


ما نیازی به ستون زمان نداریم پس بیایید اون رو حذف کنیم تا توی فضای حافظه صرفه جویی کنیم

In [7]:
ratings_df = ratings_df.drop('timestamp', axis=1)
ratings_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


# سیستم پیشنهاد دهی مبتنی بر محتوا
ما می خواهیم تلاش کنیم تا ژانرهای مورد علاقه ورودی از فیلم ها و امتیازهای داده شده رو پیدا کنیم

با ایجاد یک کاربر ورودی برای پیشنهاد فیلم ها شروع کنیم

In [8]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


## به ورودی کاربر movieid اضافه کردن
ما می‌تونیم این کار رو ابتدا با فیلتر کردن ردیف‌هایی که عنوان فیلم ورودی ما رو دارند و سپس ادغام این زیرمجموعه با دیتافریم ورودی انجام بدیم

همچنین ستون‌های غیرضروری برای ورودی رو حذف می‌کنیم تا فضای اشغال شده توی حافظه کمتر بشه


فیلتر کردن فیلم‌ها بر اساس عنوان :

In [9]:
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]

های مربوط به ورودی رو استخراج کنیم movieid سپس اون رو با اطلاعات ورودی ادغام میکنیم تا بتونیم :

In [10]:
inputMovies = pd.merge(inputId, inputMovies)

اطلاعاتی رو که از آونها استفاده نمی‌کنیم از دیتا فریم ورودی حذف میکنیم

In [11]:
inputMovies = inputMovies.drop('genres', axis=1).drop('year', axis=1)

 دیتا فریم نهایی ورودی : 

In [12]:
inputMovies

,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1274,Akira,4.5
4,1968,"Breakfast Club, The",5.0


استخراج کنیم moviesWithGenres_df های ورودی دیتا فریم مربوطه رو از movieid حالا باید طبق 

In [13]:
# جدا کردن فیلم های مربوط به ورودی 
userMovies = moviesWithGenres_df[moviesWithGenres_df['movieId'].isin(inputMovies['movieId'].tolist())]
userMovies

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
257,296,Pulp Fiction,"[Comedy, Crime, Drama, Thriller]",1994,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
973,1274,Akira,"[Action, Adventure, Animation, Sci-Fi]",1988,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1445,1968,"Breakfast Club, The","[Comedy, Drama]",1985,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


دیتا فریم رو کمی تمیز تر میکنیم year و movieid,title,genres ما فقط به جدول ژانرها نیاز داریم بنابراین با تنظیم مجدد شماره ردیف ها و حذف ستون های 

In [14]:
# تنظیم مجدد شماره ردیف ها برای جلوگیری از اشتباهات جزئی
userMovies = userMovies.reset_index(drop=True)
# حذف کردن ستون های غیر ضروری 
userGenreTable = userMovies.drop('movieId', axis=1).drop('title', axis=1).drop('genres', axis=1).drop('year', axis=1)
userGenreTable

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


تو این مرحله دیتا فریم ما اماده پیاده سازی الگوریتم است


برای انجام این کار قصد داریم هر ژانر رو به وزن‌ مربوط به اون تبدیل کنیم باید امتیاز داده شده به فیلم ها توسط کاربر رو توی تک تک ژانر های مربوط به اون فیلم ضرب کنیم 

این کار رو انجام میدیم pandas سپس مجموع امتیازات هر ژانر رو بدست بیاریم این عمل در اصل یک ضرب ماتریسی هست که به کمک کتابخونه

In [15]:
inputMovies['rating']

0    3.5
1    2.0
2    5.0
3    4.5
4    5.0
Name: rating, dtype: float64

In [16]:
# ضرب نقطه‌ای برای دریافت وزن‌ها
userProfile = userGenreTable.transpose().dot(inputMovies['rating'])
# (میزان وزن علاقه مندی کاربر به ژانر های مختلف) اطلاعات کاربر 
userProfile

Adventure             10.0
Animation              8.0
Children               5.5
Comedy                13.5
Fantasy                5.5
Romance                0.0
Drama                 10.0
Action                 4.5
Crime                  5.0
Thriller               5.0
Horror                 0.0
Mystery                0.0
Sci-Fi                 4.5
War                    0.0
Musical                0.0
Documentary            0.0
IMAX                   0.0
Western                0.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

حالا ما وزن هر ژانر مربوط به اطلاعات ورودی کاربر رو داریم با استفاده از این ما میتونیم فیلم هایی رو پیشنهاد بدیم که نیازهای کاربر رو فراهم میکنه

بیایید با استخراج جدول ژانر از جدول اصلی شروع کنیم :

In [17]:
# جدا کردن ژانر فیلم ها از دیتا فریم اصلی  
genreTable = moviesWithGenres_df.set_index(moviesWithGenres_df['movieId'])
# حذف ستون های غیر ضروری 
genreTable = genreTable.drop('movieId', axis=1).drop('title', axis=1).drop('genres', axis=1).drop('year', axis=1)
genreTable.head()

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
genreTable.shape

(9742, 20)

خب الان باید میانگین وزن دار هر فیلم رو بر اساس اطلاعات کاربر ورودی حساب کنیم و بعد 20 فیلم اول که بیشتر از همه نیاز کاربر رو برطرف میکنه پیشنهاد بدیم

In [19]:
# ضرب ژانرها با وزن‌ مربوط به هر ژانر و بعد گرفتن میانگین وزن‌دار هر فیلم  
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recommendationTable_df.head()

movieId
1    0.594406
2    0.293706
3    0.188811
4    0.328671
5    0.188811
dtype: float64

In [20]:
# موارد پیشنهادی‌مون رو به ترتیب نزولی مرتب میکنیم 
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)
recommendationTable_df.head()

movieId
134853    0.734266
148775    0.685315
117646    0.678322
6902      0.678322
81132     0.671329
dtype: float64

# جدول پیشنهاد نهایی

In [21]:
movies_df.loc[movies_df['movieId'].isin(recommendationTable_df.head(20).keys())].reset_index(drop=True)

,movieId,title,genres,year
0,673,Space Jam,"[Adventure, Animation, Children, Comedy, Fanta...",1996
1,1907,Mulan,"[Adventure, Animation, Children, Comedy, Drama...",1998
2,2987,Who Framed Roger Rabbit?,"[Adventure, Animation, Children, Comedy, Crime...",1988
3,4719,Osmosis Jones,"[Action, Animation, Comedy, Crime, Drama, Roma...",2001
4,6902,Interstate 60,"[Adventure, Comedy, Drama, Fantasy, Mystery, S...",2002
5,26340,"Twelve Tasks of Asterix, The (Les douze travau...","[Action, Adventure, Animation, Children, Comed...",1976
6,32031,Robots,"[Adventure, Animation, Children, Comedy, Fanta...",2005
7,40339,Chicken Little,"[Action, Adventure, Animation, Children, Comed...",2005
8,51939,TMNT (Teenage Mutant Ninja Turtles),"[Action, Adventure, Animation, Children, Comed...",2007
9,52287,Meet the Robinsons,"[Action, Adventure, Animation, Children, Comed...",2007
